In [ ]:
# Import modules
import numpy as np
from scipy import stats
import basic
from astropy.table import Table, vstack, hstack
from astropy.io import fits, ascii
from astropy import units as u
import warnings

#Suppress iPython warnings
warnings.filterwarnings('ignore')

#Import list of exposures/CCDs
night = Table.read(r'C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51\exposures.fits')

#Create blank array with 200*200 bins
retm3 = np.load(r'C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51\retarray.npy')*0
retm4 = np.load(r'C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51\retarray.npy')*0

#Load bin edges
xedge = np.load(r'C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51\xedgegigd200b.npy')
yedge = np.load(r'C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51\yedgegigd200b.npy')

#Load all exposures
for i in [0]:
    #Load data using basic.py
    t = basic.Catalog.read(r"C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51\20{0}\{1}\pcat\{2}".format(
        night['night'][i+i*6][1:3],night['night'][i+i*6][0:7],night['night'][i+i*6][0:14]))
    
    #Loading subscriptable catalog
    z = Table.read(r"C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51\20{0}\{1}\pcat\{2}".format(
        night['night'][i+i*6][1:3],night['night'][i+i*6][0:7],night['night'][i+i*6][0:14]))
    
    #Correct for extinction
    g_extinct = t.extinction_correct('g')
    i_extinct = t.extinction_correct('i')
    d_extinct = t.extinction_correct('ddo51')

    ##########################
    ########  CUTS  ##########
    ##########################
    
    #Cuts
    ind, = np.where((z['m_gaia']==2)&                 
    (z['m_gaia']==2))                                      # Require a Gaia DR2 match
    
    #Applying match cuts
    z = z[ind]
    g_extinct = g_extinct[ind]
    i_extinct = i_extinct[ind]
    d_extinct = d_extinct[ind]
    
    #Magnitude of less than 20 cut
    ind, = np.where(g_extinct<20)
    
    #Applying magnitude cut
    z = z[ind]
    g_extinct = g_extinct[ind]
    i_extinct = i_extinct[ind]
    d_extinct = d_extinct[ind]
    
    #Remove nans
    ind = np.where((~np.isnan(g_extinct))&(~np.isnan(i_extinct))&(~np.isnan(d_extinct)))
    
    #nan cut
    z = z[ind]
    g_extinct = g_extinct[ind]
    i_extinct = i_extinct[ind]
    d_extinct = d_extinct[ind]
        
    ##########################
    
    ###CHANGE
    #Numpy arrays with photometry
    g = np.array(g_extinct)
    ip = np.array(i_extinct)
    d = np.array(d_extinct)
    
    #Colors
    y = g - d
    x = g - ip
 
    if len(x)>0 and len(y)>0:

        ret,xed,yed,bin = stats.binned_statistic_2d(x,y,None,'count',bins=[xedge,yedge])

        retm3 = np.add(ret,retm3)

        #Removing last entry.  This will enable me to recenter the array values on the center of the bins.
        xe = xedge[0:200]
        ye = yedge[0:200]

        #Centering x and y values on center of bins
        xc = xe + (xedge[1] - xedge[0])/2
        yc = ye + (yedge[1] - yedge[0])/2

        count = retm3.T

        #Build up y values from maximum counts in each column
        ymaxcount = np.array([],dtype=int)
        for l in range(0,200):
            ycol = count[:,l]
            ymaxcount = np.append(ymaxcount,np.max(ycol))

        yidx = np.array([],dtype=int)

        for l in range(0,200):
            ycol = count[:,l]
            yt = np.where(count[:,l]==ymaxcount[l])
            yt = np.round(np.average(yt))
            yidx = np.array(np.append(yidx,yt),dtype=int)

        xcoord = xc
        ycoord = yc[yidx]

        def smooth(y, box_pts):
            box = np.ones(box_pts)/box_pts
            y_smooth = np.convolve(y, box, mode='same')
            return y_smooth

        dloc = smooth(ycoord,12)

        ind, = np.where((xcoord>0)&(xcoord<2.5))
        dloc = dloc[ind]
        xcoord = xcoord[ind]

        #Adjusted dwarf locus
        photint = np.interp(x,xcoord,dloc)
        y = y  - photint

        ##########################
        ###### GIANT CUT #########
        ##########################

        ind, = np.where((y<0.6302*(x-0.544)+0.087)&
                        (y>0.087)&
                        (y<0.45)&
                        (x<2.7)&
                        ~((y<0.14)&(x>1.0841))&
                        ~((y<0.6302*(x-1)+0.087)&(x<1.0841)&(x>1)))

        #Apply cuts to colors
        x = x[ind]
        y = y[ind]
        g = g[ind]
        ip = ip[ind]
        d = d[ind]
        photint = photint[ind]

        #Apply cuts to fits table
        z = z[ind]

        ##########################

        #Proper motions in RA and DEC
        pmra = z['pmra']
        pmde = z['pmde']

        #Calculate total proper motion
        tpm = np.sqrt(pmra**2+pmde**2)

        #fits table to house newly calculated data
        col1 = fits.Column(name='dr_g', format='D05', array=np.zeros(len(z)))
        col2 = fits.Column(name='dr_i', format='D05', array=np.zeros(len(z)))
        col3 = fits.Column(name='dr_ddo51', format='D05', array=np.zeros(len(z)))
        col4 = fits.Column(name='tpm', format='D05', array=np.zeros(len(z)))
        col5 = fits.Column(name='dloc', format='D05', array=np.zeros(len(z)))
        col6 = fits.Column(name='delta_g_d', format='D05', array=np.zeros(len(z)))
        coldefs = fits.ColDefs([col1, col2,col3,col4,col5,col6])
        dat = fits.BinTableHDU.from_columns(coldefs)
        dat = Table.read(dat)
        
        #Append data to fits table
        dat['dr_g'] = g
        dat['dr_i'] = ip
        dat['dr_ddo51'] = d
        dat['tpm'] = tpm
        dat['dloc'] = photint
        dat['delta_g_d'] = y

        #Add new data to previous fits table
        tab = hstack([z,dat])
        #tab = Table.read(tab)
        
        #Fix issue with None in units
        for l in tab.columns:
            if tab['{0}'.format(l)].unit == None:
                tab['{0}'.format(l)].unit = '-'
            else:
                pass

        #Save giants fits to file
        tab.write(r"C:\Users\shebe\OneDrive\School\Graduate\Research\Nidever\Data\DDO51\Giants\{0}.fits"
                  .format(night['night'][i+i*6][0:11]))
    
    else:
        pass